In [1]:
import sys
path_append = "../"
sys.path.append(path_append)  # Go up one directory from where you are.

import sklearn
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

import os
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch import nn
import numpy as np
import glob
import tqdm

from torch.utils.tensorboard import SummaryWriter

pd.options.mode.chained_assignment = None


c:\Users\ccn-team\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Feature Engineering

In [2]:
'''
    TRD_DD : Date
    ISU_CD : Stock Code
    ISU_NM : Stock Name
    TDD_CLSPRC : Closing Price
    TDD_OPNPRC : Opening Price
    TDD_HGPRC : High Price
    TDD_LWPRC : Low Price
    MKTCAP : Market Capitalization
    ACC_TRDVOL : Trading Volume
    EPS : Earnings Per Share
    PER : Price-Earnings Ratio
    BPS : Book Value Per Share
    PBR : Price-Book Ratio
    DPS : Dividends Per Share
    DVD_YLD : Dividend Yield

'''

'\n    TRD_DD : Date\n    ISU_CD : Stock Code\n    ISU_NM : Stock Name\n    TDD_CLSPRC : Closing Price\n    TDD_OPNPRC : Opening Price\n    TDD_HGPRC : High Price\n    TDD_LWPRC : Low Price\n    MKTCAP : Market Capitalization\n    ACC_TRDVOL : Trading Volume\n    EPS : Earnings Per Share\n    PER : Price-Earnings Ratio\n    BPS : Book Value Per Share\n    PBR : Price-Book Ratio\n    DPS : Dividends Per Share\n    DVD_YLD : Dividend Yield\n\n'

In [3]:

def load_and_merge_csv_files(data_directory, preprocessed_directory, file_limit=None):
    data_files = glob.glob(os.path.join(data_directory, "*.csv"))
    preprocessed_files = glob.glob(os.path.join(preprocessed_directory, "*.csv"))
    
    data_files = data_files[:file_limit]
    preprocessed_files = preprocessed_files[:file_limit]
    
    merged_dfs = []
    
    for data_file in data_files:
        file_name = os.path.basename(data_file)
        
        preprocessed_file_name = file_name.replace('.csv', '_preprocessed.csv')
        preprocessed_file_path = os.path.join(preprocessed_directory, preprocessed_file_name)
        
        if preprocessed_file_path in preprocessed_files:
            df_data = pd.read_csv(data_file)
            df_preprocessed = pd.read_csv(preprocessed_file_path)
            
            merged_df = pd.merge(df_data, df_preprocessed, on='TRD_DD', suffixes=('_data', '_preprocessed'))
            merged_dfs.append(merged_df)
    
    total_df = pd.concat(merged_dfs, ignore_index=True)
    
    return total_df

data_directory = path_append + "../data/KR_Data/data"
preprocessed_directory = path_append + "../data/KR_Data/preprocessed"
total_df = load_and_merge_csv_files(data_directory, preprocessed_directory)


total_df.head()


,TRD_DD,ISU_CD,ISU_NM,TDD_CLSPRC,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,MKTCAP,ACC_TRDVOL,EPS,...,BPS,PBR,DPS,DVD_YLD,GDC_sig,RSI_sig,ROC_sig,MAP_sig,STC_sig,TREND
0,2022/01/11,900110,이스트아시아홀딩스,145,150,151,143,"24,719,670,905","4,147,345",NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN
1,2022/01/10,900110,이스트아시아홀딩스,150,152,155,149,"25,572,073,350","2,628,028",NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN
2,2022/01/07,900110,이스트아시아홀딩스,151,158,162,148,"25,742,553,839","7,561,654",NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN
3,2022/01/06,900110,이스트아시아홀딩스,158,172,186,156,"26,935,917,262","22,931,278",NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN
4,2022/01/05,900110,이스트아시아홀딩스,171,164,187,161,"29,152,163,619","29,419,967",NaN,...,NaN,NaN,NaN,NaN,0,0,-1,0,0,NaN


In [4]:
# Assuming total_df is already defined and merged from previous steps

# Reverse the DataFrame to sort dates from past to present
total_df = total_df[::-1].reset_index(drop=True)

# Split the "TRD_DD" column into year, month, and day columns
total_df[["Y", "M", "D"]] = total_df["TRD_DD"].str.split("/", expand=True)

# Drop the original "TRD_DD" column
total_df = total_df.drop("TRD_DD", axis=1)

# Rearrange columns to have year, month, and day first
total_df = total_df[["Y", "M", "D"] + total_df.columns[:-3].to_list()]


In [5]:
# Create a new 'Date' column by combining 'Y', 'M', 'D' columns
total_df['Date'] = pd.to_datetime(total_df[['Y', 'M', 'D']].rename(columns={'Y': 'year', 'M': 'month', 'D': 'day'}))

# Set 'Date' as the index
total_df.set_index('Date', inplace=True)

# Create a 'count_day' column that represents the number of days from the first date
total_df['count_day'] = (total_df.index - total_df.index.min()).days

# Drop the 'Y', 'M', 'Day' columns as they're no longer needed
total_df.drop(columns=['Y', 'M', 'D'], inplace=True)

# Reorder the columns to make 'count_day' first
cols = ['count_day'] + [col for col in total_df.columns if col != 'count_day']
total_df = total_df[cols]

total_df.head()

,count_day,ISU_CD,ISU_NM,TDD_CLSPRC,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,MKTCAP,ACC_TRDVOL,EPS,...,BPS,PBR,DPS,DVD_YLD,GDC_sig,RSI_sig,ROC_sig,MAP_sig,STC_sig,TREND
Date,,,,,,,,,,,,,,,,,,,,,
2010-04-21,5468,900100,뉴프라이드(Reg.S),"12,750","15,000","17,250","12,750","89,250,000,000","2,186,517",NaN,...,NaN,NaN,NaN,NaN,0,0,-1,1,0,NaN
2010-04-22,5469,900100,뉴프라이드(Reg.S),"10,850","10,850","10,850","10,850","75,950,000,000","47,396",NaN,...,NaN,NaN,NaN,NaN,0,0,-1,1,0,NaN
2010-04-23,5470,900100,뉴프라이드(Reg.S),"9,280","9,470","10,800","9,250","64,960,000,000","5,960,215",NaN,...,NaN,NaN,NaN,NaN,0,0,-1,1,0,NaN
2010-04-26,5473,900100,뉴프라이드(Reg.S),"10,650","9,560","10,650","9,510","74,550,000,000","2,521,236",NaN,...,NaN,NaN,NaN,NaN,0,0,-1,1,0,1.000000
2010-04-27,5474,900100,뉴프라이드(Reg.S),"9,510","11,300","11,600","9,430","66,570,000,000","3,643,891",NaN,...,NaN,NaN,NaN,NaN,0,0,-1,1,0,0.981481


In [6]:
total_df.drop(['ISU_CD'], axis=1, inplace=True)

In [7]:
total_df.reset_index(drop=True, inplace=True)

In [8]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9384645 entries, 0 to 9384644
Data columns (total 20 columns):
 #   Column      Dtype  
---  ------      -----  
 0   count_day   int64  
 1   ISU_NM      object 
 2   TDD_CLSPRC  object 
 3   TDD_OPNPRC  object 
 4   TDD_HGPRC   object 
 5   TDD_LWPRC   object 
 6   MKTCAP      object 
 7   ACC_TRDVOL  object 
 8   EPS         object 
 9   PER         object 
 10  BPS         object 
 11  PBR         object 
 12  DPS         object 
 13  DVD_YLD     object 
 14  GDC_sig     int64  
 15  RSI_sig     int64  
 16  ROC_sig     int64  
 17  MAP_sig     int64  
 18  STC_sig     int64  
 19  TREND       float64
dtypes: float64(1), int64(6), object(13)
memory usage: 1.4+ GB


In [9]:
# Display non-NaN values of the columns to be dropped (for verification)
print("EPS non-NaN values:\n", total_df["EPS"].dropna())
print("PER non-NaN values:\n", total_df["PER"].dropna())
print("BPS non-NaN values:\n", total_df["BPS"].dropna())
print("PBR non-NaN values:\n", total_df["PBR"].dropna())
print("DPS non-NaN values:\n", total_df["DPS"].dropna())
print("DVD_YLD non-NaN values:\n", total_df["DVD_YLD"].dropna())

# Drop the unusable columns
total_df = total_df.drop(["EPS", "PER", "BPS", "PBR", "DPS", "DVD_YLD"], axis=1)


EPS non-NaN values:
 23765          -
23766          -
23767          -
23768          -
23769          -
           ...  
9374643    1,033
9374644    1,033
9374645    1,033
9374646    1,033
9374647    1,033
Name: EPS, Length: 9125271, dtype: object
PER non-NaN values:
 23765          -
23766          -
23767          -
23768          -
23769          -
           ...  
9374643    14.09
9374644    13.79
9374645    14.13
9374646    14.13
9374647    13.89
Name: PER, Length: 9125271, dtype: object
BPS non-NaN values:
 23765           -
23766           -
23767           -
23768           -
23769           -
            ...  
9374643    11,860
9374644    11,860
9374645    11,860
9374646    11,860
9374647    11,860
Name: BPS, Length: 9125271, dtype: object
PBR non-NaN values:
 23765         -
23766         -
23767         -
23768         -
23769         -
           ... 
9374643    1.23
9374644    1.20
9374645    1.23
9374646    1.23
9374647    1.21
Name: PBR, Length: 9125271, dtype: object


In [10]:
import pandas as pd

# Assuming total_df is already defined and filled with NaN values replaced by 0
# total_df = ...

# 1) Set TREND to 0 for any value that is not -1, 0, or 1
total_df.loc[~total_df["TREND"].isin([-1, 0, 1]), "TREND"] = 0

# 2) Set TREND to -1 for negative values and 1 for positive values
total_df.loc[total_df["TREND"] < 0, "TREND"] = -1
total_df.loc[total_df["TREND"] > 0, "TREND"] = 1

# 3) Adjust TREND values based on the specified conditions
total_df.loc[total_df["TREND"] <= -0.5, "TREND"] = -1
total_df.loc[total_df["TREND"] >= 0.5, "TREND"] = 1
total_df.loc[(total_df["TREND"] > -0.5) & (total_df["TREND"] < 0.5), "TREND"] = 0

# Check the unique values in the TREND column and their counts
unique_trends = set(total_df["TREND"])
trend_counts = total_df["TREND"].value_counts()

# Print the unique values and their counts
print("Unique TREND values:", unique_trends)
print("TREND value counts:\n", trend_counts)



Unique TREND values: {0.0, 1.0, -1.0}
TREND value counts:
  0.0    8933593
 1.0     225720
-1.0     225332
Name: TREND, dtype: int64


In [11]:
total_df["TREND"] += 1

In [12]:
total_df["TREND"] = total_df["TREND"].convert_dtypes(int)
total_df["TREND"]

0          1
1          1
2          1
3          2
4          1
          ..
9384640    1
9384641    1
9384642    1
9384643    1
9384644    1
Name: TREND, Length: 9384645, dtype: Int64

In [13]:
import pandas as pd

# List of columns to convert from strings to numeric values
columns_to_convert = ["TDD_CLSPRC", "TDD_OPNPRC", "TDD_HGPRC", "TDD_LWPRC", "MKTCAP", "ACC_TRDVOL"]

# Convert the columns to numeric values
for col in columns_to_convert:
    total_df[col] = total_df[col].str.replace(pat=r'[^0-9]', repl=r'' ,regex=True).apply(pd.to_numeric)



In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, RobustScaler

# Assuming total_df is already defined and filled with NaN values replaced by 0
# total_df = ...

# Define the scalers
mm = MinMaxScaler()
sc = RobustScaler()

# # Apply MinMax scaling to the specified columns
# minmax_cols = ["count_day", "TDD_CLSPRC", "TDD_OPNPRC", "TDD_HGPRC", "TDD_LWPRC"]
# for col in minmax_cols:
#     total_df[col] = mm.fit_transform(total_df[col].values.reshape(-1, 1))

# # Apply Robust scaling to the specified columns
# robust_cols = ["MKTCAP", "ACC_TRDVOL"]
# for col in robust_cols:
#     total_df[col] = sc.fit_transform(total_df[col].values.reshape(-1, 1))


In [ ]:
total_df.head()

In [ ]:
total_df.info()

In [ ]:
# Ensure 'ISU_NM' is of string type
total_df["ISU_NM"] = total_df["ISU_NM"].astype(str)

# Calculate where 'ISU_NM' column changes value
isu_nm_changes = total_df['ISU_NM'].shift() != total_df['ISU_NM']
change_indices = [0] + isu_nm_changes[isu_nm_changes].index.tolist() + [len(total_df)]

# Compute pairs of (start, end) indices
segment_pairs = [(change_indices[i], change_indices[i+1]) for i in range(len(change_indices) - 1)]

print("Pairs of (start, end) indices:", segment_pairs)

In [ ]:
# Drop ISU_NM if it exists
if "ISU_NM" in total_df.columns:
    total_df = total_df.drop("ISU_NM", axis=1)
else:
    print("Column 'ISU_NM' not found in DataFrame")

In [ ]:
import dask.dataframe as dd

# Convert to Dask DataFrame
ddf = dd.from_pandas(total_df, npartitions=10)

# Apply pd.to_numeric in a distributed manner
ddf_numeric = ddf.map_partitions(lambda df: df.apply(pd.to_numeric, errors='coerce')).compute()

ddf_numeric.info()


In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler

def convert_nullable_int_columns(df):

    int_columns = df.select_dtypes(include=['Int64']).columns
    for col in int_columns:
        df[col] = df[col].astype('int64')
    return df

def gpu_standard_scaler(tensor, dim = 1):
    return (tensor - tensor.mean(dim = dim))/(tensor.std(dim = dim) + 1e-8)


def process_dataframe(df, segments, use_scale=False, include_diff=False):

    df_numeric = df.apply(pd.to_numeric, errors='coerce')

    df_numeric = convert_nullable_int_columns(df_numeric)

    df_numeric = df_numeric.dropna()

    df_tensor = torch.tensor(df_numeric.values, dtype=torch.float64).cuda()

    df_list_diff = []

    for start, end in segments:
        segment = df_tensor[start:end]

        if use_scale:
            segment = gpu_standard_scaler(segment, dim = 0)
            # scaler = StandardScaler()
            # segment = torch.tensor(scaler.fit_transform(segment.cpu()), dtype=torch.float64).cuda()

        if include_diff:
            segment_diff = segment[1:] - segment[:-1]
            df_list_diff.append(segment_diff)

    if include_diff:
        processed_tensor = torch.cat(df_list_diff, dim=0)
    else:
        processed_tensor = torch.cat([df_tensor[start:end] for start, end in segments], dim=0)

    processed_df = pd.DataFrame(processed_tensor.cpu().numpy(), columns=df_numeric.columns)

    new_segment_pairs = [(0, len(segment)) for segment in df_list_diff] if include_diff else segments

    return processed_df, new_segment_pairs

total_df, segment_pairs = process_dataframe(total_df, segment_pairs, use_scale=True, include_diff=False)


In [ ]:
segment_pairs

# Data Overview and Usage Guide

In [ ]:
# Data Overview and Usage Guide

"""
- Data Overview
    Preprocessed Data: total_df
    Categorical Columns: Y, M, D, ISU_CD, GDC_sig, RSI_sig, ROC_sig, MAP_sig, STC_sig
    Numerical Columns: TDD_CLSPRC, TDD_OPNPRC, TDD_HGPRC, TDD_LWPRC, MKTCAP, ACC_TRDVOL
    Label: TREND

- Considerations:
    1) It is recommended to use embedding techniques for categorical data.
    2) Labels:
        NaN values have been replaced with 0.

        2-1) Label Processing:
            How to handle -1, 0, 1 depends on the definition.
            ● Classification of -1, 0, 1:
                Commonly, the label being discrete is an issue.
                (1) Set to -1 for values less than 0, and 1 for values greater than 0.
                    # Ratio of -1, 0, 1 = 1397:1440:55
                    : This results in very frequent trading.

                (2) Use only -1, 0, 1.
                    # Ratio of -1, 0, 1 = 76:2740:76
                    : This might cause the model to miss buying opportunities when it should, making it difficult for the model to make accurate predictions.

                (3) Set to -1 for values less than -0.5, and 1 for values greater than 0.5, otherwise 0.
                    # Ratio of -1, 0, 1 = 752:1409:731
                    : (Current preprocessing state) This provides a somewhat balanced ratio.

            ● Regression:
                Keep the label as it is.
                (1) The model performs regression and decides whether to buy or sell based on the predicted increase or decrease.

    3) The utility of GDC, RSI, ROC, MAP, STC indicators for learning is uncertain.
"""


In [ ]:
import torch
import random
import torch.nn.functional as F
from torch.utils.data import Dataset

class SequentialDataset(Dataset):
    def __init__(self, df, segment_pairs, max_window_size):
        self.df = df
        self.segment_pairs = segment_pairs
        self.max_window_size = max_window_size
        self.min_window_size = max_window_size // 2
        
        # Compute the total number of possible subsequences
        self.subsequence_lengths = [
            max(0, end - start - self.min_window_size + 1)
            for start, end in self.segment_pairs
        ]
        self.cumulative_lengths = [sum(self.subsequence_lengths[:i+1]) for i in range(len(self.subsequence_lengths))]

    def __len__(self):
        return self.cumulative_lengths[-1]

    def __getitem__(self, idx):
        # Find the appropriate segment for the given idx
        segment_idx = next(i for i, cum_len in enumerate(self.cumulative_lengths) if cum_len > idx)
        
        # Adjust idx to be relative to the found segment
        if segment_idx > 0:
            idx -= self.cumulative_lengths[segment_idx - 1]
        
        start_idx, end_idx = self.segment_pairs[segment_idx]
        actual_start_idx = start_idx + idx
        actual_end_idx = next(end for start, end in self.segment_pairs if start <= actual_start_idx < end)
        
        window_size = random.randint(self.min_window_size, self.max_window_size)
        window_size = min(window_size, actual_end_idx - actual_start_idx)
        
        seq_end_idx = min(actual_start_idx + window_size, actual_end_idx)

        seq = self.df.iloc[actual_start_idx:seq_end_idx]

        X = seq.drop(['TREND'], axis=1)
        y = seq['TREND']

        X = torch.tensor(X.values, dtype=torch.float32)

        label = torch.tensor(y.values.astype(int), dtype=torch.long)
        label = F.one_hot(label, num_classes=3)

        return X, label


In [ ]:
from random import shuffle

# Assuming 'df' and 'num_classes' are defined
max_window_size = 64
shuffle(segment_pairs)  # Shuffle the indices to randomize the data order
train_indices, test_indices = segment_pairs[:int(0.8 * len(segment_pairs))], segment_pairs[int(0.8 * len(segment_pairs)):]

trainset = SequentialDataset(df=total_df, indices=train_indices, max_window_size=max_window_size)
testset = SequentialDataset(df=total_df, indices=test_indices, max_window_size=max_window_size)

print('Train indices: ', len(train_indices))
print('Test indices: ', len(test_indices))

print(trainset[0][0].shape)
print(len(trainset))

In [ ]:
from tools.setting.ml_params import MLParameters
from tools.setting.data_config import DataConfig
from nn.utils.init import set_random_seed
set_random_seed(0)

In [ ]:
data_config = DataConfig(dataset_name = 'stock_price', task_type='multi_class_classification', obs_shape=[12], label_size=3)

#  Set training configuration from the AlgorithmConfig class, returning them as a Namespace object.
ml_params = MLParameters(core_model = 'gpt', encoder_model = 'none')

first_data = trainset[0]
X, y = first_data

print(f"Input shape: {X.shape}")
print(f"Label shape: {y.shape}")

print(f"Total number of samples in trainset: {len(trainset)}")

In [ ]:
from trainer_hub import TrainerHub

# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Initialize the TrainerHub class with the training configuration, data configuration, device, and use_print and use_wandb flags
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=True, use_wandb=False)

In [ ]:
trainer_hub.train(trainset, testset)